In [2]:
import os 
import json 

import numpy as np 
from pathlib import Path

In [3]:
scoring_experiments_path = Path("./experiments/FINAL_SCORING_EXPERIMENTS/")

experiments = os.listdir(scoring_experiments_path)
experiments

['2025-04-27_ScoringExperiment____home__stud__n__nr063__mounted_home__mad_gan_thesis__code__experiments__CMADGAN_MODELS_DATACREATION__FASHION__CMADGAN_10_GEN_DataCreation_SPEC_GEN_0',
 '2025-04-27_ScoringExperiment____home__stud__n__nr063__mounted_home__mad_gan_thesis__code__experiments__CMADGAN_MODELS_DATACREATION__FASHION__CMADGAN_10_GEN_DataCreation_SPEC_GEN_1',
 '2025-04-27_ScoringExperiment____home__stud__n__nr063__mounted_home__mad_gan_thesis__code__experiments__CMADGAN_MODELS_DATACREATION__FASHION__CMADGAN_10_GEN_DataCreation_SPEC_GEN_2',
 '2025-04-27_ScoringExperiment____home__stud__n__nr063__mounted_home__mad_gan_thesis__code__experiments__CMADGAN_MODELS_DATACREATION__FASHION__CMADGAN_10_GEN_DataCreation_SPEC_GEN_3',
 '2025-04-27_ScoringExperiment____home__stud__n__nr063__mounted_home__mad_gan_thesis__code__experiments__CMADGAN_MODELS_DATACREATION__FASHION__CMADGAN_10_GEN_DataCreation_SPEC_GEN_4',
 '2025-04-27_ScoringExperiment____home__stud__n__nr063__mounted_home__mad_gan_th

In [4]:
def load_json_data(path): 
    with open(path, "r") as f: 
        data = json.load(f)

    d = {
        "fid_score": data["fid_score"],
        "is_score": data["is_score"][0],
        "is_score_std": data["is_score"][1],
        
    }
    return d 

def avg_experiment_scores(dicts): 
    fid = []
    iss = []
    isstd = []
    

    for d in dicts: 
        fid.append(d["fid_score"])
        iss.append(d["is_score"])
        isstd.append(d["is_score_std"])

    return {
        "fid_score": np.mean(fid),
        "is_score": np.mean(iss),
        "is_score_std": np.mean(isstd)
    }
    
def dict_of_dicts_to_latex_table(data: dict, caption: str = "My Table", label: str = "tab:my_table") -> str:
    """
    Convert a dictionary of dictionaries into a LaTeX table string.

    Args:
        data (dict): Dictionary where each key is a row name and each value is a dictionary of column entries.
        caption (str): LaTeX caption for the table.
        label (str): LaTeX label for referencing the table.

    Returns:
        str: LaTeX table code as a string.
    """
    if not data:
        return ""

    # Extract headers from the first row
    headers = list(next(iter(data.values())).keys())

    # Escape underscores in headers
    headers_escaped = [h.replace('_', r'\_') for h in headers]

    # Start LaTeX table
    latex = []
    latex.append(r"\begin{table}[ht]")
    latex.append(r"    \centering")
    latex.append(r"    \begin{tabular}{" + "|l" + "|c" * len(headers) + "|}")
    latex.append(r"    \hline")
    
    # Column names
    header_row = "Row & " + " & ".join(headers_escaped) + r" \\"
    latex.append("    " + header_row)
    latex.append(r"    \hline")

    

    # Data rows
    for row_name, row_data in data.items():
        row_values = [row_data.get(col, "") for col in headers]

        
        # Round numbers if they are floats
        formatted_values = []
        for v in row_values:
            if isinstance(v, float):
                formatted_values.append(f"{v:.3f}")
            else:
                formatted_values.append(str(v))
                
        
        row_line = f"{row_name.replace('_', ' ')} & $" + "$ & $".join(formatted_values) + r"$ \\"
        latex.append("    " + row_line)

    latex.append(r"    \hline")
    latex.append(r"    \end{tabular}")
    latex.append(f"    \\caption{{{caption}}}")
    latex.append(f"    \\label{{{label}}}")
    latex.append(r"\end{table}")

    return "\n".join(latex)


In [18]:
madgan_fashion = {}
cmadgan_fashion = {}
madgan_mnist = {}
cmadgan_mnist = {}
cond_fashion = {}
cond_mnist = {}
dc_fashion = {} 
dc_mnist ={}

for exp in experiments: 
    exp = exp.lower()
    if "_madgan_" in exp and "_mnist_" in exp: 
        madgan_mnist[exp[-30:].strip("_")] = load_json_data(scoring_experiments_path / exp / "output.json")
    elif "fashionmnist_madgan_" in exp: 
        madgan_fashion[exp[-30:].strip("_")] = load_json_data(scoring_experiments_path / exp / "output.json")
    elif "cmadgan" in exp and "mnist" in exp: 
        cmadgan_mnist[exp[-30:].strip("_")] = load_json_data(scoring_experiments_path / exp / "output.json")
    elif "fashion__cmadgan" in exp: 
        cmadgan_fashion[exp[-30:].strip("_")] = load_json_data(scoring_experiments_path / exp / "output.json")
    elif "vanilla" in exp and "fashion" in exp: 
        dc_fashion[exp[-30:].strip("_")] = load_json_data(scoring_experiments_path / exp / "output.json")
    elif "vanilla" in exp and "mnist" in exp: 
        dc_mnist[exp[-30:].strip("_")] = load_json_data(scoring_experiments_path / exp / "output.json")
    elif "conditional" in exp and "fashion" in exp: 
        cond_fashion[exp[-30:].strip("_")] = load_json_data(scoring_experiments_path / exp / "output.json") 
    elif "conditional" in exp and "mnist" in exp: 
        cond_mnist[exp[-30:].strip("_")] = load_json_data(scoring_experiments_path / exp / "output.json")
    else: 
        print(f"Could not identify experiment: {exp}")
        
    


In [19]:
for d in [
    madgan_fashion,
    cmadgan_fashion,
    madgan_mnist,
    cmadgan_mnist,
    cond_fashion,
    cond_mnist,
    dc_fashion,
    dc_mnist,
]: 
    print(len(d.keys()))

25
25
25
25
1
1
1
1


In [20]:
for v, _d in {
    "madgan_fashion": madgan_fashion,
    "cmadgan_fashion": cmadgan_fashion,
    "madgan_mnist": madgan_mnist,
    "cmadgan_mnist": cmadgan_mnist,
    "cond_fashion": cond_fashion,
    "cond_mnist": cond_mnist,
    "dc_fashion": dc_fashion,
    "dc_mnist": dc_mnist,
}.items():  
    print()
    print(v)
    print(dict_of_dicts_to_latex_table(_d))
    print()


madgan_fashion
\begin{table}[ht]
    \centering
    \begin{tabular}{|l|c|c|c|}
    \hline
    Row & fid\_score & is\_score & is\_score\_std \\
    \hline
    10 gen datacreation spec gen 0 & $22.454$ & $4.534$ & $0.117$ \\
    10 gen datacreation spec gen 1 & $21.914$ & $4.524$ & $0.121$ \\
    10 gen datacreation spec gen 2 & $21.327$ & $4.494$ & $0.089$ \\
    10 gen datacreation spec gen 3 & $21.630$ & $4.455$ & $0.063$ \\
    10 gen datacreation spec gen 4 & $20.851$ & $4.575$ & $0.097$ \\
    10 gen datacreation spec gen 5 & $21.268$ & $4.561$ & $0.048$ \\
    10 gen datacreation spec gen 6 & $22.324$ & $4.566$ & $0.127$ \\
    10 gen datacreation spec gen 7 & $21.633$ & $4.542$ & $0.084$ \\
    10 gen datacreation spec gen 8 & $21.503$ & $4.533$ & $0.127$ \\
    10 gen datacreation spec gen 9 & $20.963$ & $4.552$ & $0.100$ \\
    3 gen datacreation spec gen 0 & $26.086$ & $4.463$ & $0.103$ \\
    3 gen datacreation spec gen 1 & $25.884$ & $4.487$ & $0.099$ \\
    3 gen datacreat

In [24]:
from collections import defaultdict
import re

def compute_grouped_averages(results_dict):
    grouped = defaultdict(lambda: {'fid': [], 'is': [], 'is_std': []})
    
    for key, metrics in results_dict.items():
        match = re.match(r'([^\_]+)', key)
        if match:
            group = match.group(1)
            grouped[group]['fid'].append(metrics['fid_score'])
            grouped[group]['is'].append(metrics['is_score'])
            grouped[group]['is_std'].append(metrics['is_score_std'])

    averages = {}
    for group, vals in grouped.items():
        averages[group] = {
            'avg_fid': sum(vals['fid']) / len(vals['fid']),
            'avg_is': sum(vals['is']) / len(vals['is']),
            'avg_is_std': sum(vals['is_std']) / len(vals['is_std']),
        }

    return averages


In [29]:
for v, d in {
    "madgan_fashion": madgan_fashion,
    "cmadgan_fashion": cmadgan_fashion,
    "madgan_mnist": madgan_mnist,
    "cmadgan_mnist": cmadgan_mnist,
    "cond_fashion": cond_fashion,
    "cond_mnist": cond_mnist,
    "dc_fashion": dc_fashion,
    "dc_mnist": dc_mnist,
}.items(): 
    print(v)
    di = compute_grouped_averages(d)
    for k, v in di.items(): 
        print(k)
        print(f"${round(v['avg_fid'], 3)}$ & ${round(v['avg_is'], 3)}$ & ${round(v['avg_is_std'], 3)}$ \\\\")
    print()
    print()

madgan_fashion
10
$21.587$ & $4.534$ & $0.097$ \\
3
$26.202$ & $4.496$ & $0.099$ \\
5
$24.218$ & $4.497$ & $0.098$ \\
7
$23.875$ & $4.523$ & $0.094$ \\


cmadgan_fashion
10
$159.067$ & $3.317$ & $0.035$ \\
3
$25.555$ & $4.623$ & $0.105$ \\
5
$160.082$ & $3.346$ & $0.038$ \\
7
$154.115$ & $2.929$ & $0.034$ \\


madgan_mnist
3
$23.177$ & $2.511$ & $0.04$ \\
5
$22.656$ & $2.471$ & $0.044$ \\
10
$20.973$ & $2.474$ & $0.037$ \\
7
$21.599$ & $2.533$ & $0.051$ \\


cmadgan_mnist
10
$110.553$ & $2.062$ & $0.018$ \\
3
$25.578$ & $2.398$ & $0.036$ \\
5
$29.071$ & $2.35$ & $0.039$ \\
7
$30.645$ & $2.354$ & $0.039$ \\


cond_fashion
shion
$123.349$ & $3.573$ & $0.117$ \\


cond_mnist
mnist
$122.097$ & $2.611$ & $0.056$ \\


dc_fashion
nerative
$25.56$ & $4.21$ & $0.099$ \\


dc_mnist
nerative
$28.721$ & $2.553$ & $0.022$ \\


